In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.data.pipes.map.mux

In [4]:
# export
# Python native modules
import os
from inspect import isfunction,ismethod
from typing import *
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
# from torch.utils.data.dataloader import DataLoader as OrgDataLoader
import torchdata.datapipes as dp
from torch.utils.data.dataloader_experimental import DataLoader2
from fastai.data.transforms import *
# Local modules

# Basic DataPipes - Multiplexer
> Basic datapipes for work with fastrl core API.

In [5]:
#For example, so not exported

from fastai.vision.core import *
from fastai.vision.data import *
from fastai.data.external import *

untar_data(URLs.MNIST_TINY),URLs.MNIST_TINY

(Path('/home/fastrl_user/.fastai/data/mnist_tiny'),
 'https://s3.amazonaws.com/fast-ai-sample/mnist_tiny.tgz')

Load the mnist csv...

In [6]:
pipe = dp.iter.IterableWrapper([str(untar_data(URLs.MNIST_TINY)/'labels.csv')]) # FileOpener really should support Path as well as str
pipe = dp.iter.FileOpener(pipe, mode="b")
pipe = dp.iter.CSVParser(pipe,skip_lines=1)

class AddIdx():
    def __init__(self): self.idx=0
    def __call__(self,file):
        try:     return (self.idx,file)
        finally: self.idx+=1

base_pipe = dp.map.IterToMapConverter(pipe,key_value_fn=AddIdx())
pipe = dp.map.IterToMapConverter(pipe,key_value_fn=AddIdx())
# pipe[5],len(base_pipe)

In [7]:
len(pipe)

/opt/conda/lib/python3.7/site-packages/torchdata/datapipes/iter/util/converter.py:97: UserWarning: Data from prior DataPipe are loaded to get length ofIterToMapConverter before execution of the pipeline.Please consider removing len().
  "Data from prior DataPipe are loaded to get length of"


1408

Now that we have the csv converted into a map, we want to split it into a training and validation dataset...

In [8]:
from fastrl.fastai.data.pipes.map.demux import *

ImportError: cannot import name '_check_lambda_fn' from 'torch.utils.data.datapipes.utils.common' (/opt/conda/lib/python3.7/site-packages/torch/utils/data/datapipes/utils/common.py)

In [ ]:
def train_valid_splitter(o): 
    
    int_mapping={'train':0,'valid':1}
    
    return int_mapping[Path(o[0]).parts[0]]


dp1, dp2 = DemultiplexerMapDataPipe(pipe,num_instances=2, classifier_fn=train_valid_splitter, drop_none=True)

In [ ]:
assert len(dp1)+len(dp2)==len(pipe),f"The demux'd dp1 and dp2 when added together should be the same len as pipe {len(dp1)} + {len(dp2)} = {len(pipe)}"

In [ ]:
dp1[0],dp2[100]

In [ ]:
list(dp1)[:5],list(dp2)[:5]

In [ ]:
class KSplitter():
    k=0
    def __init__(self,k_splits=2): self.k_splits=k_splits
    def __call__(self,*args):
        try: 
            return self.k
        finally: 
            self.k+=1
            if self.k==self.k_splits: self.k=0

k1,k2,k3 = DemultiplexerMapDataPipe(dp1,num_instances=3,  classifier_fn=KSplitter(k_splits=3), drop_none=True)

In [ ]:
# export
from itertools import chain, zip_longest

from typing import Dict, Iterable, Optional

from torch.utils.data.datapipes._decorator import functional_datapipe
from torch.utils.data.datapipes.datapipe import MapDataPipe


@functional_datapipe("mux")
class MultiplexerMapDataPipe(MapDataPipe):
    def __init__(self, *datapipes, dp_index_map: Optional[Dict[MapDataPipe, Iterable]] = None):
        self.datapipes = datapipes
        self.dp_index_map = dp_index_map if dp_index_map else {}
        self.length: Optional[int] = None
        self.index_map = {}
        # Create a generator that yields (index, (dp_num, old_index)) in sequentially order.
        indices = (self._add_dp_num(i, dp) for i, dp in enumerate(datapipes))
        dp_id_and_key_tuples = chain.from_iterable(zip_longest(*indices))
        self.key_gen = enumerate(e for e in dp_id_and_key_tuples if e is not None)

    def _add_dp_num(self, dp_num: int, dp: MapDataPipe):
        # Assume 0-index for all DataPipes unless alternate indices are defined in `self.dp_index_map`
        dp_indices = self.dp_index_map[dp] if dp in self.dp_index_map else range(len(dp))
        for idx in dp_indices:
            yield dp_num, idx

    def __getitem__(self, index):
        if 0 <= index < len(self):
            if index in self.index_map:
                dp_num, old_key = self.index_map[index]
            else:
                curr_key = -1
                while curr_key < index:
                    curr_key, dp_num_key_tuple = next(self.key_gen)
                    dp_num, old_key = dp_num_key_tuple
                self.index_map[index] = dp_num, old_key
            try:
                return self.datapipes[dp_num][old_key]
            except KeyError:
                raise RuntimeError(
                    f"Incorrect key is given to MapDataPipe {dp_num} in Multiplexer, likely because"
                    f"that DataPipe is not 0-index but alternate indices are not given."
                )
        raise RuntimeError(f"Index {index} is out of bound for Multiplexer.")

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

    def __len__(self):
        if self.length is None:
            self.length = 0
            for dp in self.datapipes:
                self.length += len(dp)
        return self.length

In [ ]:
a = dp.map.SequenceWrapper(range(10))
b = dp.map.SequenceWrapper({'a': 100, 'b': 200, 'c': 300, 'd': 400})
datapipe = a.mux(b, dp_index_map={b: ['a', 'b', 'c', 'd']})
list(datapipe)  # Returns [0, 100, 1, 200, 2, 300, 3, 400, 4, 5, 6, 7, 8, 9]

In [ ]:
len(k1),len(k2),len(k3)

In [ ]:
combined_pipes=MultiplexerMapDataPipe(k1,k2,k3)

In [ ]:
list(combined_pipes)[::100]

In [ ]:
len(combined_pipes)

In [ ]:
combined_pipes[5]

In [ ]:
list(combined_pipes)[::100]

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)

In [ ]:
# # export
# dp.functional_datapipe('mux')
# class MultiplexerMapDataPipe(dp.map.MapDataPipe):
#     def __init__(self, *datapipes):
#         self.datapipes = datapipes
#         # datapipe import standard is import torchdata as dp. We need to make these
#         # private
#         for _dp in self.datapipes:
#             if not isinstance(_dp,dp.map.MapDataPipe):
#                 dp_types=[type(o) for o in self.datapipes]
#                 raise ValueError(f'Passed in datapipes need to be MapDataPipes, got {dp_types}')
        
#         self.length: Optional[int] = None
#         self._map = {}

#     def _setup_datapipe_indexer(self, datapipe) -> Optional[Iterator[Any]]:
#         # self._datapipe_iterator: Optional[Iterator[Any]] = None
#         # Instead of _datapipe_iterator we have _datapipe_indexer
#         # We need to know how to get the index from the main_datapipe. In order
#         # to do this, we check if it is...
        
#         # NOTE: THIS IS NOT A GOOD SOLUTION SINCE THIS CANT RELY ON A STANDARD
#         # INTERFACE FOR GETTING INDEXES
        
#         # We cash the indexes because we want to be able to have consistent behavior 
#         # when calling __getitem__ on a child pipe. 
#         # What we don't want is the main_datapipe being indexed by `str` but the
#         # child pipes indexing by `int`...
#         if isinstance(datapipe, dp.map.SequenceWrapper):
#             return iter(range(len(datapipe)))
#         elif hasattr(datapipe, '_map'):
#             return iter(datapipe._map)
#         elif hasattr(datapipe, 'index_map'):
#             return iter(datapipe.index_map)
#         else:
#             warnings.warn('data pipe will be indexed by len')
#             return iter(range(len(datapipe)))
        
#     def __iter__(self):
#         iterators = [self._setup_datapipe_indexer(x) for x in self.datapipes]
#         finished: Set[int] = set()
#         while len(finished) < len(iterators):
#             for i in range(len(iterators)):
#                 if i not in finished:
#                     try:
#                         index = next(iterators[i])
#                         # print(i,index)
#                         value = self.datapipes[i][index]
#                         # self._map will track which index is associated with 
#                         # which datapipe...
#                         self._map[index] = i
#                         yield value
#                     except StopIteration:
#                         finished.add(i)

#     def __getitem__(self, index) -> T_co:
#         if index in self._map:
#             # self._map[index] -> the datapipe to getitem at, then pass index to 
#             # get the value 
#             return self.datapipes[self._map[index]][index]
        
#         # Remember that iter(self) adds index to self._map. So as we iter,
#         # we can check if index has be found in a datapipe, and once found, return 
#         # that value.
#         for value in self:
#             if index in self._map: return value
        
#         raise IndexError(f'Unable to find {index} in the datapipes')
                        
#     def __len__(self):
#         if self.length is not None:
#             if self.length == -1:
#                 raise TypeError("{} instance doesn't have valid length".format(type(self).__name__))
#             return self.length
#         if all(isinstance(_dp, Sized) for _dp in self.datapipes):
#             self.length = sum(len(_dp) for _dp in self.datapipes)
#         else:
#             self.length = -1
#         return len(self)
